In [8]:
import akshare as ak
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

import db

# code_list = ak.stock_info_a_code_name()
df = pd.DataFrame(columns=['total_val', 'stock_id', 'category', 'stock_name'])
# print(code_list['code'][0])
stock_individual_info_em_df = ak.stock_individual_info_em(symbol="000001")
df_transposed = stock_individual_info_em_df.set_index('item').T.reset_index()
df_transposed = df_transposed.rename(columns={
    '总市值': 'total_val',
    '行业': 'category',
    '股票代码': 'stock_id',
    '股票简称': 'stock_name'
})

df = pd.concat([df, df_transposed[['total_val', 'stock_id', 'category', 'stock_name']]], ignore_index=True)


db_username = 'root'
db_password = 'qwertyui'
db_host = 'localhost'  # 或者其他主机地址
db_port = '3306'
db_name = 'schemas'
table_name = 'stock_info'  # 要创建或写入的表的名称

# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# 将数据写入 MySQL
df.to_sql(table_name, engine, if_exists='replace', index=False)

# 关闭数据库连接
engine.dispose()

In [7]:
import akshare as ak
stock_zh_a_hist_min_em_df = ak.stock_zh_a_hist_min_em(symbol="000001", start_date="2022-11-24 09:30:00", end_date="2023-11-25 09:30:00", period='1', adjust='')
print(stock_zh_a_hist_min_em_df)

                       时间    开盘     收盘     最高     最低    成交量         成交额  \
0     2023-11-20 09:30:00   0.0  10.15  10.15  10.15   7628   7742420.0   
1     2023-11-20 09:31:00   0.0  10.14  10.18  10.14  17726  18002691.0   
2     2023-11-20 09:32:00   0.0  10.15  10.16  10.14   7517   7624925.0   
3     2023-11-20 09:33:00   0.0  10.13  10.15  10.13   6391   6480819.0   
4     2023-11-20 09:34:00   0.0  10.14  10.14  10.13   5254   5323795.0   
...                   ...   ...    ...    ...    ...    ...         ...   
1200  2023-11-24 14:56:00  10.1  10.10  10.11  10.10   4852   4900930.0   
1201  2023-11-24 14:57:00  10.1  10.11  10.11  10.10   2884   2913855.0   
1202  2023-11-24 14:58:00  10.1  10.10  10.10  10.10    900    909000.0   
1203  2023-11-24 14:59:00  10.1  10.10  10.10  10.10      0         0.0   
1204  2023-11-24 15:00:00  10.1  10.10  10.10  10.10   8781   8868810.0   

         最新价  
0     10.150  
1     10.154  
2     10.152  
3     10.150  
4     10.148  
...      

In [39]:
import pandas as pd
import pymysql
from datetime import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import akshare as ak

# 获取数据
data = ak.stock_bid_ask_em(symbol="000001")
data_transposed = data.set_index('item').T.reset_index()
data_transposed['stock_id'] = "000001"
data_transposed['time'] = datetime.now().timestamp()
data_transposed.drop(columns='index', inplace=True)

# 数据库配置
db_username = 'root'
db_password = 'qwertyui'
db_host = 'localhost'
db_port = '3306'
db_name = 'schemas'
table_name = 'stock_bid'

# 创建数据库连接
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# 创建会话
Session = sessionmaker(bind=engine)
session = Session()

# 获取数据并处理
data = ak.stock_bid_ask_em(symbol="000001")
data_transposed = data.set_index('item').T.reset_index()
data_transposed['stock_id'] = "000001"
data_transposed['time'] = datetime.now().timestamp()
data_transposed.drop(columns='index', inplace=True)

# 执行删除和插入操作
for index, row in data_transposed.iterrows():
    # 查询是否存在
    existing_data = session.query(StockBid).filter_by(stock_id=row['stock_id']).first()
    if existing_data:
        session.delete(existing_data)
    new_data = StockBid(stock_id=row['stock_id'], ...)
    session.add(new_data)

session.commit()
session.close()

SyntaxError: positional argument follows keyword argument (331269335.py, line 40)

In [41]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="002838", period="daily", start_date="20231120", end_date='20231124', adjust="")
print(stock_zh_a_hist_df)

           日期     开盘     收盘     最高     最低    成交量          成交额    振幅   涨跌幅  \
0  2023-11-20  13.63  13.62  13.70  13.49  20731  28155062.00  1.54  0.07   
1  2023-11-21  13.62  13.64  13.80  13.60  23623  32355003.24  1.47  0.15   
2  2023-11-22  13.59  13.51  13.73  13.50  21763  29578591.88  1.69 -0.95   
3  2023-11-23  13.46  13.61  13.64  13.46  19327  26248644.60  1.33  0.74   
4  2023-11-24  13.61  13.49  13.64  13.44  18765  25341356.35  1.47 -0.88   

    涨跌额   换手率  
0  0.01  0.53  
1  0.02  0.61  
2 -0.13  0.56  
3  0.10  0.50  
4 -0.12  0.48  


In [17]:
import db
db.fetch_min_data()

Exception in thread Thread-32:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 954, in _bootstrap_inner
Exception in thread Thread-31:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 892, in run
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mac/Downloads/myflask/db.py", line 73, in get_min_data
    with lock:
NameError: name 'engine' is not defined
    self._target(*self._args, **self._kwargs)
  File "/Users/mac/Downloads/myflask/db.py", line 73, in get_min_data
    with lock:
NameError: name 'engine' is not defined
Exception in thread Thread-30:
Traceback (most recent 

In [ ]:
import akshare as ak
import pandas as pd
import threading
from sqlalchemy import create_engine
from datetime import datetime, timedelta
db_username = 'root'
db_password = 'qwertyui'
db_host = 'localhost'
db_port = '3306'
db_name = 'schemas'
table_name = 'stock_info'

# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

sql = f"SELECT * FROM {table_name}"


lock = threading.Lock()
monitor = 0


In [45]:
import db
code = "000001"
data = ak.stock_zh_a_hist(symbol=code,period="daily",start_date="20221120",end_date="20231119",adjust="")
data = data.rename(columns={
    '日期': 'date',
    '开盘': 'open_price',
    '收盘': 'close_price',
    '最高': 'high_price',
    '最低': 'low_price',
    '成交量': 'volume',
    '成交额': 'turnover',
    '振幅': 'amplitude',
    '涨跌幅': 'price_change_pct',
    '涨跌额': 'price_change_amt',
    '换手率': 'turnover_rate'
})
data['stock_id']=code
db.pd2sql(data,"stock_hist")
